# Practica con métodos de agrupamiento basados en densidad y en probabilidad con este notebook en Python
En este *notebook* aprenderás a ejecutar un método de agrupamiento basado en densidad (DBSCAN) y otro basado en probabilidad (GMM/Expectation-Maximization) utilizando las librerías *PyClustering* y *scikit-learn*. Veremos cómo configurar sus parámetros y visualizar sus resultados.

## 1. Ejecutar el algoritmo DBSCAN en ***PyClustering***
En la primera parte del *notebook* utilizaremos la librería *PyClustering*. En concreto, nos centraremos en el paquete [*cluster.dbscan*](https://pyclustering.github.io/docs/0.8.2/html/d2/d42/classpyclustering_1_1cluster_1_1dbscan_1_1dbscan.html), donde se encuentra la implementación del método DBSCAN. Como en notebooks anteriores, primero tenemos que importar los paquetes que necesitaremos:

In [ ]:
# La primera vez que se vaya a ejecutar este notebook es necesario instalar la librería pyclustering
!pip install pyclustering
from pyclustering.cluster.dbscan import dbscan
from pyclustering.cluster import cluster_visualizer
import numpy as np
import matplotlib.pyplot as plt

Vamos a utilizar el *dataset* Iris para este ejemplo, ya que viene incorporado en la librería. De sus cuatro propiedades numéricas, nos quedaremos con las dos primeras a las que llamaremos *x* e *y*. Representamos esta muestra bi-dimensional con *matplotlib* para ver la distribución de los datos.

In [ ]:
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import FAMOUS_SAMPLES
datos = read_sample(FAMOUS_SAMPLES.SAMPLE_IRIS)
x = [punto[0] for punto in datos]
y = [punto[1] for punto in datos]
datos_xy = np.column_stack((x, y))
print(x)
print(y)
plt.scatter(x, y)
plt.show()

A continuación, podemos configurar los parámetros del algoritmo DBSCAN. Tenemos que especificar el valor de *eps* (radio de vecindad) y el de *neighbors* (número mínimo de vecinos).

In [ ]:
eps = 0.25
neighbors = 5
alg_dbscan = dbscan(datos_xy, eps, neighbors);

Para ejecutar el análisis de grupos, invocamos a la función *process* como hacíamos con otros métodos de esta misma librería.

In [ ]:
alg_dbscan.process()

Una vez ejecutado el algoritmo, podemos obtener los grupos por medio del método *get_clusters*. En este caso, no sabemos de antemano cuántos grupos se han obtenido, pero podemos averiguarlo con el método *len*.

In [ ]:
# Devuelve un array de k elementos, donde cada elemento continene el índice de las instancias asignadas al grupo k
grupos = alg_dbscan.get_clusters()
print(grupos)
num_grupos = len(grupos)
print(num_grupos)

Además, el algoritmo DBSCAN aisla los puntos considerados como "ruido" o "outliers". Para obtenerlos, utilizamos el método *get_noise*.

In [ ]:
ruido = alg_dbscan.get_noise()
print(ruido)
num_outliers = len(ruido)
print(num_outliers)

Podemos visualizar la asignación en una gráfica bidimensional utilizando la clase *visualizer*.

In [ ]:
grafico = cluster_visualizer();
grafico.append_clusters(grupos, datos_xy)
grafico.show();

## 2. Ejecutar el algoritmo Expectation-Maximization en ***PyClustering***
Vamos ahora a trabjar con el agrupamiento basado en probabilidad que nos ofrece *PyClustering*. Se trata del modelo de mezcla de distribuciones gaussianas con el algoritmo *Expectation-Maximization*. Este algoritmo está en el paquete [*cluster.ema*](https://pyclustering.github.io/docs/0.8.2/html/d1/d24/namespacepyclustering_1_1cluster_1_1ema.html)

In [ ]:
from pyclustering.cluster.ema import ema, ema_initializer, ema_observer, ema_visualizer, ema_init_type

Según su [documentación](https://pyclustering.github.io/docs/0.8.2/html/d4/d22/classpyclustering_1_1cluster_1_1ema_1_1ema__init__type.html), podemos seleccionar el tipo de inicialización entre aleatoria o basada en *k-means++*. En este caso, vamos a utilizar la forma de inicialización no aleatoria, por lo que primero necesitamos estimar los valores iniciales de medias y la matriz de covarianzas. 

In [ ]:
num_grupos = 3
medias_inicial, covarianzas_inicial = ema_initializer(datos_xy, num_grupos).initialize(ema_init_type.KMEANS_INITIALIZATION)
print(medias_inicial)
print(covarianzas_inicial)

A continuación, podemos configurar el método con la inicialización obtenida y ejecutar el análisis de grupos. En la primera línea, estamos creando un objeto de tipo *observer* que nos va a permitir ver cómo varía el proceso de agrupamiento.

In [ ]:
alg_ema = ema(datos_xy, num_grupos, medias_inicial, covarianzas_inicial);
alg_ema.process()

Tras ejecutarlo, podemos extraer no solo los grupos, sino también las medias y covarianzas finales.



In [ ]:
grupos = alg_ema.get_clusters()
print(grupos)
medias_final = alg_ema.get_centers()
print(medias_final)
covarianzas_final = alg_ema.get_covariances()
print(covarianzas_final)

Por último, vamos a visualizar de forma dinámica cómo se han ido definiendo las distribuciones de probabilidad alrededor de los datos.

In [ ]:
grafico = ema_visualizer.show_clusters(grupos, datos_xy, covarianzas_final, medias_final, display=False)
grafico.show()

## 3. Ejecutar el algoritmo DBSCAN en ***scikit-learn***
Vamos a utilizar el mismo conjunto de datos (Iris) pero ahora utilizaremos la implementación disponible en [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). Para ello, primero importamos el algoritmo.

In [ ]:
from sklearn.cluster import DBSCAN

Vamos a utilizar la misma muestra de datos (datos_xy) y los mismos parámetros que en la ejecución con *PyClustering*.

In [ ]:
alg_dbscan_sklearn = DBSCAN(eps=0.25, min_samples=5)
alg_dbscan_sklearn.fit(datos_xy)

Como DBSCAN asigna -1 a los datos etiquetados como ruido, debemos procesar por separado estos puntos. Por ejemplo, para conocer el número de grupos identificado podemos utilizar la siguiente línea de código:

In [ ]:
etiquetas_dbscan = alg_dbscan_sklearn.labels_
print(etiquetas_dbscan)
num_grupos = len(set(etiquetas_dbscan)) - (1 if -1 in etiquetas_dbscan else 0)
print(num_grupos)

Para conocer el número de puntos etiquetados como ruido, simplemente tenemos que contar el número de veces que aparece la etiqueta -1:

In [ ]:
num_outliers = list(etiquetas_dbscan).count(-1)
print(num_outliers)

## 4. Ejecutar el algoritmo Expectation-Maximization en ***scikit-learn***
En *scikit-learn* también podemos aplicar el método EM asociado al modelo de mezclas gaussianas. Puedes encontrar su documentación en el paquete [sklearn.mixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture):

In [ ]:
from sklearn.mixture import GaussianMixture

En su configuración, podemos indicar el número de componentes gaussianas a estimar y la semilla para la inicialización aleatoria.

In [ ]:
alg_gmm_sklearn = GaussianMixture(n_components=3, random_state=0, init_params='random')
alg_gmm_sklearn.fit(datos_xy)

Para evitar los inconvenientes de la inicialización aleatoria, podemos aplicar alguna de estas dos opciones:


1.   Generar más de una inicialización aleatoria, quedándonos con la mejor. Para ello, se debe utilizar el parámetro *n_init*.
2.   Utilizar el método k-means para inicializar, que de hecho es el valor por defecto del parámetro *init_params*.



In [ ]:
alg_gmm_sklearn = GaussianMixture(n_components=3, random_state=0, init_params='kmeans')
alg_gmm_sklearn.fit(datos_xy)

GaussianMixture(n_components=3, random_state=0)

Una vez ejecutado el método, podemos acceder a sus propiedades: pesos de cada componente del *mixture model*, medias, y covarianzas

In [ ]:
pesos = alg_gmm_sklearn.weights_
print(pesos)
medias = alg_gmm_sklearn.means_
print(medias)
covarianzas = alg_gmm_sklearn.covariances_
print(covarianzas)

Como es habitual en *sklearn*, podemos usar el método *predict* para predecir a qué grupo se asignaría una nueva muestra.

In [ ]:
nuevo_dato = [[7,3.5]]
prediccion = alg_gmm_sklearn.predict(nuevo_dato)
print(prediccion)